In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer


In [2]:
import numpy as np 
import pandas as pd 

submission_3 = pd.read_csv("superblend_1.csv")




In [3]:

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('kaggle_proj_Lakshmi/train.csv').fillna('No data')
test = pd.read_csv('kaggle_proj_Lakshmi/test.csv').fillna('No data')

#train.id = train.id.astype('str')
#train.comment_text = train.comment_text.astype('str')

#test.id = test.id.astype('str')
#test.comment_text = test.comment_text.astype('str')


In [4]:
submission_3['comment_text'] = test.comment_text
submission_3.shape


(153164, 8)

In [21]:
new = submission_3.copy()
new = new[['id', 'comment_text']]
new['toxic'] = [1 if x >= 0.5 else 0 for x in submission_3.toxic]
new['severe_toxic'] = [1 if x >= 0.5 else 0 for x in submission_3.severe_toxic]
new['obscene'] = [1 if x >= 0.5 else 0 for x in submission_3.obscene]
new['threat'] = [1 if x >= 0.5 else 0 for x in submission_3.threat]
new['insult'] = [1 if x >= 0.5 else 0 for x in submission_3.insult]
new['identity_hate'] = [1 if x >= 0.5 else 0 for x in submission_3.identity_hate]

In [22]:
stacked = pd.concat([train, new])
stacked.shape

(312735, 8)

In [7]:
train_text = train[['id', 'comment_text']]
test_text = test[['id', 'comment_text']]
combined_text = stacked[['id', 'comment_text']]

In [55]:
#imp_text1

In [8]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    encoding='utf-8',
    lowercase=True,
    min_df=0.00001,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    #use_idf=1, smooth_idf=1,
    max_features=None)

word_vectorizer.fit(combined_text.comment_text)
train_word_features = word_vectorizer.transform(stacked.comment_text)
test_word_features = word_vectorizer.transform(test_text.comment_text)

msg = "There are {} tokens in Comment_text if we use word"
print(msg.format(len(word_vectorizer.get_feature_names())))

There are 62311 tokens in Comment_text if we use word


In [24]:
#train_features = hstack([train_char_features, train_word_features])
#test_features = hstack([test_char_features, test_word_features])
#X_train, X_valid, y_train, y_valid = train_test_split(train_features, train_target, test_size=0.3, random_state=42)


scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = stacked[class_name]
    classifier = LogisticRegression(solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_word_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_word_features, train_target)
    submission[class_name] = classifier.predict_proba(test_word_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))
print submission.head()
submission.to_csv('mysubmission4.csv', index=False)

CV score for class toxic is 0.97719047262
CV score for class severe_toxic is 0.990332264938
CV score for class obscene is 0.986147514504
CV score for class threat is 0.980374583097
CV score for class insult is 0.978111173144
CV score for class identity_hate is 0.973003191267
Total CV score is 0.980859866595
                 id     toxic  severe_toxic   obscene    threat    insult  \
0  00001cee341fdb12  0.990884      0.087384  0.948575  0.008777  0.649078   
1  0000247867823ef7  0.003902      0.001251  0.002033  0.000766  0.003216   
2  00013b17ad220c46  0.020768      0.001975  0.008696  0.000840  0.010336   
3  00017563c3f7919a  0.001701      0.001113  0.001781  0.000586  0.002650   
4  00017695ad8997eb  0.020130      0.001018  0.004389  0.001021  0.005544   

   identity_hate  
0       0.064072  
1       0.001654  
2       0.002055  
3       0.000575  
4       0.001254  


In [22]:
#train_features = hstack([train_char_features, train_word_features])
#test_features = hstack([test_char_features, test_word_features])

scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = stacked[class_name]
    X_train, X_valid, y_train, y_valid = train_test_split(train_word_features, train_target, test_size=0.3, random_state=42)


    classifier = LogisticRegression(solver='sag')

    cv_score = np.mean(cross_val_score(classifier, X_valid, y_valid, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(X_train, y_train)
    #submission[class_name] = classifier.predict_proba(test_word_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

#submission.to_csv('F:\impwordssub.csv', index=False)

CV score for class toxic is 0.980953488273
CV score for class severe_toxic is 0.985811848279
CV score for class obscene is 0.99076922687
CV score for class threat is 0.982669141613
CV score for class insult is 0.980363178041
CV score for class identity_hate is 0.979922374566
Total CV score is 0.983414876274


In [9]:
#train_features = hstack([train_char_features, train_word_features])
#test_features = hstack([test_char_features, test_word_features])

scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = stacked[class_name]
    X_train, X_valid, y_train, y_valid = train_test_split(train_word_features, train_target, test_size=0.3, random_state=42)


    classifier = LogisticRegression(solver='sag')

    cv_score = np.mean(cross_val_score(classifier, X_valid, y_valid, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(X_train, y_train)
    #submission[class_name] = classifier.predict_proba(test_word_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

#submission.to_csv('F:\impwordssub.csv', index=False)

CV score for class toxic is 0.981490249653
CV score for class severe_toxic is 0.988404743899
CV score for class obscene is 0.990830873049
CV score for class threat is 0.985799281
CV score for class insult is 0.981899729415
CV score for class identity_hate is 0.982936343245
Total CV score is 0.985226870044


In [62]:
#train_features = hstack([train_char_features, train_word_features])
#test_features = hstack([test_char_features, test_word_features])

scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = stacked[class_name]
    X_train, X_valid, y_train, y_valid = train_test_split(train_word_features, train_target, test_size=0.3, random_state=42)


    classifier = MultinomialNB()

    cv_score = np.mean(cross_val_score(classifier, X_valid, y_valid, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(X_train, y_train)
    #submission[class_name] = classifier.predict_proba(test_word_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

#submission.to_csv('F:\impwordssub.csv', index=False)

CV score for class toxic is 0.946554920876
CV score for class severe_toxic is 0.936215919315
CV score for class obscene is 0.9355287361
CV score for class threat is 0.886947003746
CV score for class insult is 0.93722629103
CV score for class identity_hate is 0.914459874538
Total CV score is 0.926155457601


In [10]:
nrow = submission.shape[0]
nrow

153164

In [11]:
#0.4 - all

73.9135828262516
72.8271656525032
78.09602778720848
76.55388994802956
72.5346687211094
74.48943615993315


In [32]:
#0.5 -a ll

71.11201065524536

73.05045572066543

75.54255569193805

66.69125904259487

72.0639314721475

In [16]:
print (submission.toxic > submission_3.toxic).sum() * 100.0/nrow
print (submission.severe_toxic > submission_3.severe_toxic).sum() * 100.0/nrow
print (submission.obscene > submission_3.obscene).sum() * 100.0/nrow
print (submission.threat > submission_3.threat).sum() * 100.0/nrow
print (submission.insult > submission_3.insult).sum() * 100.0/nrow
print (submission.identity_hate > submission_3.identity_hate).sum() * 100.0/nrow

67.15154997257841
72.82455407275862
73.05176151053773
75.54190279700191
66.69125904259487
72.06458436708365


In [19]:
a = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
a

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [20]:

for i in a:
    new = submission_3.copy()
    new = new[['id', 'comment_text']]
    new['toxic'] = [1 if x > i else 0 for x in submission_3.toxic]
    new['severe_toxic'] = [1 if x > i else 0 for x in submission_3.severe_toxic]
    new['obscene'] = [1 if x > i else 0 for x in submission_3.obscene]
    new['threat'] = [1 if x > i else 0 for x in submission_3.threat]
    new['insult'] = [1 if x > i else 0 for x in submission_3.insult]
    new['identity_hate'] = [1 if x > i else 0 for x in submission_3.identity_hate]
    stacked = pd.concat([train, new])
    scores = []
    submission = pd.DataFrame.from_dict({'id': test['id']})
    for class_name in class_names:
        train_target = stacked[class_name]
        classifier = LogisticRegression(solver='sag')

        cv_score = np.mean(cross_val_score(classifier, train_word_features, train_target, cv=3, scoring='roc_auc'))
        scores.append(cv_score)
        print('CV score for class {} is {}'.format(class_name, cv_score))

        classifier.fit(train_word_features, train_target)
        submission[class_name] = classifier.predict_proba(test_word_features)[:, 1]

    print('Total CV score is {}'.format(np.mean(scores)))
    #print submission.head()
#submission.to_csv('mysubmission5.csv', index=False)
#calcuaitng the differences
    print (submission.toxic > submission_3.toxic).sum() * 100.0/nrow
    print (submission.severe_toxic > submission_3.severe_toxic).sum() * 100.0/nrow
    print (submission.obscene > submission_3.obscene).sum() * 100.0/nrow
    print (submission.threat > submission_3.threat).sum() * 100.0/nrow
    print (submission.insult > submission_3.insult).sum() * 100.0/nrow
    print (submission.identity_hate > submission_3.identity_hate).sum() * 100.0/nrow

CV score for class toxic is 0.980116340304
CV score for class severe_toxic is 0.989187863459
CV score for class obscene is 0.990446982521
CV score for class threat is 0.992291470522
CV score for class insult is 0.982473016854
CV score for class identity_hate is 0.984439740257
Total CV score is 0.986492568986
91.24206732652581
92.24556684338356
96.01407641482332
87.47225196521376
95.54595074560602
90.68645373586483
CV score for class toxic is 0.984276468095
CV score for class severe_toxic is 0.991735028311
CV score for class obscene is 0.99284920974
CV score for class threat is 0.991464208727
CV score for class insult is 0.98534382777
CV score for class identity_hate is 0.988503637282
Total CV score is 0.989028729987
86.11618918283669
82.25758011020866
90.77067718262776
81.40098195398396
88.69381839074457
82.97445875009794
CV score for class toxic is 0.985867154156
CV score for class severe_toxic is 0.992231170471
CV score for class obscene is 0.993664754104
CV score for class threat is